In [13]:
import os

In [14]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict'

In [15]:
os.chdir("../")

In [16]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects'

In [17]:
#Update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    Local_data_file: Path
    unzip_dir: Path

In [18]:
#Update the configuration manager in src config
from Autopredictor.src.constants import *
from Autopredictor.src.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            Local_data_file=config.Local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [20]:
#update components
import os
import urllib.request as request
import zipfile
from Autopredictor.src.logging import logger
from Autopredictor.src.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.Local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.Local_data_file
            )
            logger.info(f"{filename} Download! with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.Local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.Local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-23 00:45:55,485: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/config/config.yaml loaded successfully]
[2024-05-23 00:45:55,489: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/params.yaml loaded successfully]
[2024-05-23 00:45:55,536: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/AutoPredict/schema.yaml loaded successfully]
[2024-05-23 00:45:55,541: INFO:common: created directory at: artifacts]
[2024-05-23 00:45:55,543: INFO:common: created directory at: artifacts/data_ingestion]
[2024-05-23 00:45:57,337: INFO:1347631616: artifacts/data_ingestion/Autopredict_cleaned_data.zip Download! with the following info: 
Connection: close
Content-Length: 1952735
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9dcc43c4be27a92f997ff62396609f38e4b236e502cc2377489ebfb0f94e4330"